## Import & Load Data

In [ ]:
#halooooooooooo

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import skew

df = pd.read_csv("train.csv")
dft = pd.read_csv("test.csv")

## EDA pre-Feature Engineering

In [ ]:
#Pengaturan Tampilan Data
pd.set_option('display.max_columns', None)

In [53]:
#Cek Data yang Kosong
def missing(df) :
    missing = df.isnull().sum()
    percent = (df.isnull().sum()/len(df))*100
    missing_table = pd.DataFrame({
        'MISSING':missing,
        '%':percent
    })
    missing_table = missing_table[missing_table['MISSING'] > 0]
    missing_table['%'] = missing_table['%'].apply(lambda x: f"{x:.2f}%")
    missing_table = missing_table.sort_values(by='%', ascending=False)
    print(missing_table.round(2))
    print("\n")
    cols_with_missing = missing_table.index.tolist()
    print("Kolom yang punya missing values:", cols_with_missing)
def miss(col):
    for name, data in [("df", df), ("dft", dft)]:
        if col not in data.columns:
            print(f"Kolom '{col}' gak ada di {name}.")
            continue

        total = len(data)
        missing_count = data[col].isnull().sum()
        missing_percent = (missing_count / total) * 100

        print(f"\n[{name.upper()}]")
        print(f"Total baris        : {total}")
        print(f"Missing di '{col}' : {missing_count} ({missing_percent:.2f}%)")

    print("\nDone cek missing untuk kolom:", col)

miss('MiscVal')


[DF]
Total baris        : 1460
Missing di 'MiscVal' : 0 (0.00%)

[DFT]
Total baris        : 1459
Missing di 'MiscVal' : 0 (0.00%)

Done cek missing untuk kolom: MiscVal


In [23]:
df['Utilities'].describe()

count       1460
unique         2
top       AllPub
freq        1459
Name: Utilities, dtype: object

In [ ]:
#Reset Pengaturan Tampilan Data
pd.reset_option('display.max_rows')

In [ ]:
#Visualisasi Outlier (LotFrontage, MasVnrArea)
sns.boxplot(y='LotFrontage', data=df)
plt.show()
sns.boxplot(y='MasVnrArea', data=df)
plt.show()

In [ ]:
#Visualisasi Outlier (GrLivArea vs SalePrice)
plt.figure(figsize=(10, 6))
sns.scatterplot(x=df['GrLivArea'], y=df['SalePrice'])
plt.xlabel('GrLivArea')
plt.ylabel('SalePrice')
plt.title('Deteksi Outlier GrLivArea vs SalePrice')
plt.show()

## Hapus Outlier

In [ ]:

outlier_indices = df[(df['GrLivArea'] > 4000) & (df['SalePrice'] < 300000)].index
print(f"Dropping {len(outlier_indices)} outlier(s)...")
df = df.drop(outlier_indices)
df = df.reset_index(drop=True)

## Fill n Encode like a smartass

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import skew

df = pd.read_csv("train.csv")
dft = pd.read_csv("test.csv")

# -----MSSubClass udah di getdummies-----

#MSZoning
df['MSZoning'] = df['MSZoning'].fillna('RL') #selebihnya di getdummies

#---LotFrontage
#isi data yang kosong dengan median neighborhood
LotFrontage_median = df.groupby('Neighborhood')['LotFrontage'].median()
df['LotFrontage'] = df['LotFrontage'].fillna(df['Neighborhood'].map(LotFrontage_median))
dft['LotFrontage'] = dft['LotFrontage'].fillna(df['Neighborhood'].map(LotFrontage_median))
#---LotFrontage

#LotArea dibiarkan apa adanya

# -----Street udah di getdummies-----

#---Alley
df['Alley'] = df['Alley'].fillna('NoAlley')
dft['Alley'] = dft['Alley'].fillna('NoAlley')
#selebihnya di getdummies
#---Alley

lotshape_fin_map = {
    'IR3' : 0,
    'IR2' : 1,
    'IR1' : 2,
    'Reg' : 3,
    'NA'  : 0
}
df['LotShape'] = df['LotShape'].map(lotshape_fin_map)
dft['LotShape'] = dft['LotShape'].map(lotshape_fin_map)

# -----LandContour udah di getdummies-----

#---Utilities
df.drop('Utilities', axis=1, inplace=True)
#didrop karena 99% AllPub

# -----LotConfig sudah di getdummies

#---LandSlope
landslope_fin_map = {
    'Gtl' : 2,
    'Mod' : 1,
    'Sev' : 0
}
df['LandSlope'] = df['LandSlope'].map(landslope_fin_map)
dft['LandSlope'] = dft['LandSlope'].map(landslope_fin_map)

# -----Neighborhood udah di getdummies-----
# -----Condition1 udah di getdummies-----
# -----Condition2 udah di getdummies-----
# -----BldgType udah di getdummies-----
# -----HouseStyle udah di getdummies-----
# -----OverallQual dibiarkan apa adanya-----
# -----OverallCond dibiarkan apa adanya-----
# -----YearBuilt dibiarkan apa adanya-----
# -----YearRemodAdd dibiarkan apa adanya-----
# -----RoofStyle masuk ke getdummies-----
# -----RoofMatl masuk ke getdummies-----

#---Exterior1st dan 2nd
dft['Exterior1st'] = dft['Exterior1st'].fillna(df['Exterior1st'].mode()[0])
dft['Exterior2nd'] = dft['Exterior2nd'].fillna(df['Exterior2nd'].mode()[0])
#selebihnya di getdummies

#---MasVnrType
df['MasVnrType'] = df['MasVnrType'].fillna('None')
dft['MasVnrType'] = dft['MasVnrType'].fillna('None')
#selebihnya di getdummies

#---MasVnrArea
df['MasVnrArea'] = df['MasVnrArea'].fillna(0)
dft['MasVnrArea'] = dft['MasVnrArea'].fillna(0)
#selebihnya dibiarkan apa adanya

# -----ExterQual sudah di QC-----
# -----ExterCond sudah di QC-----
# -----Foundation di getdummies-----
# -----BsmtQual sudah di QC-----
# -----BsmtCond sudah di QC-----

#---BsmtExposure
bsmt_exp_map = {
    'Gd': 4,
    'Av': 3,
    'Mn': 2,
    'No': 1,
    'NA': 0
}
df['BsmtExposure'] = df['BsmtExposure'].fillna('NA')
dft['BsmtExposure'] = dft['BsmtExposure'].fillna('NA')
df['BsmtExposure'] = df['BsmtExposure'].map(bsmt_exp_map)
dft['BsmtExposure'] = dft['BsmtExposure'].map(bsmt_exp_map)

#---BsmtFinSF2
df['BsmtFinSF2'] = df['BsmtFinSF2'].fillna(0)
dft['BsmtFinSF2'] = dft['BsmtFinSF2'].fillna(0)
#selebihnya dibiarkan

#---BsmtFinType1
bsmt_fin_map = {
    'GLQ': 6,
    'ALQ': 5,
    'BLQ': 4,
    'Rec': 3,
    'LwQ': 2,
    'Unf': 1,
    'NA': 0
}
df['BsmtFinType1'] = df['BsmtFinType1'].fillna('NA')
dft['BsmtFinType1'] = dft['BsmtFinType1'].fillna('NA')

df['BsmtFinType1'] = df['BsmtFinType1'].map(bsmt_fin_map)
dft['BsmtFinType1'] = dft['BsmtFinType1'].map(bsmt_fin_map)

#---BsmtFinSF1
dft['BsmtFinSF1'] = dft['BsmtFinSF1'].fillna(0)
#selebihnya dibiarkan

#---BsmtFinType2
df['BsmtFinType2'] = df['BsmtFinType2'].fillna('NA')
dft['BsmtFinType2'] = dft['BsmtFinType2'].fillna('NA')
df['BsmtFinType2'] = df['BsmtFinType2'].map(bsmt_fin_map)
dft['BsmtFinType2'] = dft['BsmtFinType2'].map(bsmt_fin_map)

#---BsmtFinSF2
dft['BsmtFinSF1'] = dft['BsmtFinSF1'].fillna(0)
#selebihnya dibiarkan

#---BsmtUnfSF
dft['BsmtUnfSF'] = dft['BsmtUnfSF'].fillna(0)

#---TotalBsmtSF
dft['TotalBsmtSF'] = dft['TotalBsmtSF'].fillna(0)

# -----Heating sudah di getdummies-----
# -----HeatingQC sudah di QC-----

#---CentralAir
df['CentralAir'] = df['CentralAir'].map({'Y': 1, 'N': 0})
dft['CentralAir'] = dft['CentralAir'].map({'Y': 1, 'N': 0})

electrical_map = {
    'FuseP': 1,   # poor
    'FuseF': 2,   # fair
    'FuseA': 3,   # average
    'Mix': 4,     # mixed
    'SBrkr': 5   # standard breakers & Romex
}
df['Electrical'] = df['Electrical'].fillna('SBrkr')
dft['Electrical'] = dft['Electrical'].fillna('SBrkr')
df['Electrical'] = df['Electrical'].map(electrical_map)
dft['Electrical'] = dft['Electrical'].map(electrical_map)

#--- 1stFlrSF dibiarkan apa adanya
#--- 2ndFlrSF dibiarkan apa adanya
#--- LowQualFinSF dibiarkan apa adanya
#--- GrLivArea dibiarkan apa adanya

#---BsmtFullBath
dft['BsmtFullBath'] = dft['BsmtFullBath'].fillna(0)

#---BsmtHalfBath
dft['BsmtHalfBath'] = dft['BsmtHalfBath'].fillna(0)

#---FullBath dibiarkan apa adanya
#---HalfBath dibiarkan apa adanya
#---Bedroom dibiarkan apa adanya
#---Kitchen dibiarkan apa adanya
# -----KitchenQual udah di QS-----
#---TotRmsAbvGrd dibiarkan apa adanya

#---Functional
functional_map = {
    'Typ': 7,
    'Min1': 6,
    'Min2': 5,
    'Mod': 4,
    'Maj1': 3,
    'Maj2': 2,
    'Sev': 1,  
    'Sal': 0
}
df['Functional'] = df['Functional'].fillna(df['Functional'].mode()[0])
dft['Functional'] = dft['Functional'].fillna(df['Functional'].mode()[0])
df['Functional'] = df['Functional'].map(functional_map)
dft['Functional'] = dft['Functional'].map(functional_map)

#---Fireplaces dibiarkan apa adanya
# -----FireplaceQu udah di QS-----

#---GarageType
df['GarageType'] = df['GarageType'].fillna('NA')
dft['GarageType'] = dft['GarageType'].fillna('NA')
#selebihnya di getdummies

#---GarageYrBlt
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(0)
dft['GarageYrBlt'] = dft['GarageYrBlt'].fillna(0)

#---GarageFinish
garage_fin_map = {
    'Fin': 3,
    'RFn': 2,
    'Unf': 1,
    'NA': 0
}
df['GarageFinish'] = df['GarageFinish'].fillna('NA')
dft['GarageFinish'] = dft['GarageFinish'].fillna('NA')

df['GarageFinish'] = df['GarageFinish'].map(garage_fin_map)
dft['GarageFinish'] = dft['GarageFinish'].map(garage_fin_map)

#---GarageCars
dft['GarageCars'] = dft['GarageCars'].fillna(0)

#---GarageArea
dft['GarageArea'] = dft['GarageArea'].fillna(0)

# -----GarageQual udah di QS-----
# -----GarageCond udah di QS-----

#---PavedDrive
paveddrive_map = {
    'Y': 2,
    'P': 1,
    'N': 0
}
df['PavedDrive'] = df['PavedDrive'].map(paveddrive_map)
dft['PavedDrive'] = dft['PavedDrive'].map(paveddrive_map)

#---WoodDeckSF dibiarkan
#---OpenPorchSF dibiarkan
#---EnclosedPorch dibiarkan
#---3SsnPorch dibiarkan
#---ScreenPorch dibiarkan
#---PoolArea dibiarkan

# -----PoolQC udah di QS-----

#---Fence
fence_map = {
    'GdPrv': 4,
    'MnPrv': 3,
    'GdWo': 2,
    'MnWw': 1,
    'NA': 0
}
df['Fence'] = df['Fence'].fillna('NA')
dft['Fence'] = dft['Fence'].fillna('NA')
df['Fence'] = df['Fence'].map(fence_map)
dft['Fence'] = dft['Fence'].map(fence_map)



#---MiscFeature
df['MiscFeature'] = df['MiscFeature'].fillna('None')
dft['MiscFeature'] = dft['MiscFeature'].fillna('None')
#selebihnya di getdummies

#---MiscVal dibiarkan

#---MoSold
df["MoSold"] = df["MoSold"].fillna(df["MoSold"].mode()[0])

#---YrSold
df["YrSold"] = df["YrSold"].fillna(df["YrSold"].mode()[0])

#---SaleType
df['SaleType'] = df['SaleType'].fillna(df['SaleType'].mode()[0])
dft['SaleType'] = dft['SaleType'].fillna(df['SaleType'].mode()[0])
#selebihnya di getdummies

#---SaleCondition
df['SaleCondition'] = df['SaleCondition'].fillna(df['SaleCondition'].mode()[0])
dft['SaleCondition'] = dft['SaleCondition'].fillna(df['SaleCondition'].mode()[0])
#selebihnya di getdummies

# -------------------------QC
qual_cond_map = {
    'Ex': 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po': 1,
    'NA': 0 
}
qual_cond_cols = [
    'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 
    'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC'
]
for col in qual_cond_cols:
    df[col] = df[col].fillna("NA")
    dft[col] = dft[col].fillna("NA")
for col in qual_cond_cols:
    df[col] = df[col].map(qual_cond_map)
    dft[col] = dft[col].map(qual_cond_map)
#--------------------------QC

#--------------------------getdummies
getdummies_cols = ['MSSubClass','MSZoning','Alley','Street','LandContour',
                   'LotConfig','Neighborhood','Condition1','Condition2',
                   'BldgType','HouseStyle','RoofStyle','RoofMatl',
                   'Exterior1st','Exterior2nd','MasVnrType','Foundation'
                   ,'Heating','MiscFeature','SaleType','SaleCondition','GarageType']
df[getdummies_cols] = df[getdummies_cols].astype(str)
dft[getdummies_cols] = dft[getdummies_cols].astype(str)
df = pd.get_dummies(df, columns=getdummies_cols, drop_first=True)
dft = pd.get_dummies(dft, columns=getdummies_cols, drop_first=True)
#--------------------------getdummies

#done
#alhamdulillah
# real kejar tayang
#iya anjer aowkoawko
#up igs

## EDA

In [ ]:
# HeatMap
df_num = df.select_dtypes(include=['int64', 'float64'])
corr = df_num.corr()
plt.figure(figsize=(21, 22))
sns.heatmap(corr, annot=True, cmap='coolwarm')

In [ ]:
#Visualisasi Hubungan GarageCars, GarageArea, dan SalePrice dalam Scatterplot Matrix.
sns.pairplot(df[['GarageCars', 'GarageArea', 'SalePrice']], hue='SalePrice',palette="viridis")
plt.show()

## Transformasi dan Scaling

In [ ]:
#Log Transform SalePrice
df['SalePrice'] = np.log1p(df['SalePrice'])

#Ambil semua kolom numerik dan log yang skewed
numeric_cols = df.select_dtypes(include = [np.number]).columns
numeric_cols = numeric_cols.drop(['Id', 'SalePrice'])
skewed_feats = df[numeric_cols].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("Skewness fitur numerik:")
print(skewed_feats)
skewness_threshold = 0.75
highly_skewed_cols = skewed_feats[abs(skewed_feats) > skewness_threshold].index
print(f"Menerapkan log-transform ke {len(highly_skewed_cols)} fitur...")
for col in highly_skewed_cols:
    df[col] = np.log1p(df[col])
    dft[col] = np.log1p(dft[col])

from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
dft[numeric_cols] = scaler.transform(dft[numeric_cols])

